In [ ]:
import os
import random
import re
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

# здесь управляем контекстом беседы
class Context:
    def __init__(self):
        self.messages = []
        self.agents = {}

    def register_agent(self, agent):
        self.agents[agent.name] = agent
    
    def add(self, sender_name, content):
        self.messages.append({
            "role": "user",
            "content": f"#{sender_name}# {content}"
        })
    
    def add_response(self, agent_name, content):
        self.messages.append({
            "role": "assistant", 
            "content": f"#{agent_name}# {content}"
        })

    def get_participants(self):
        return ", ".join(self.agents.keys())


class AgentModel:
    def __init__(self, name, system_prompt, model_name, shared_context):
        self.name = name
        self.system_prompt = system_prompt
        self.model_name = model_name
        self.context = shared_context
        self.client = OpenAI(
            api_key=os.getenv("OPENAI_API_KEY"),
            base_url=os.getenv("OPENAI_BASE_URL", "https://openrouter.ai/api/v1")
        )
        shared_context.register_agent(self)
    
    def respond(self):
        messages = self.context.messages
        participants = self.context.get_participants()
        
        # системный промпт чтобы иишки хорошо общались, без него вообще ужас 
        system_prompt = f"""ты агент {self.name}
                твоя роль: {self.system_prompt}
                
                участники чата: {participants}.

                ты участвуешь  дискуссии
                читай ВСЮ историю переписки, а не только последнее сообщение
                твоя задача: спорить, иронизировать, исполнять свою роль, проявлять эмоции
                
                В ПЕРВУЮ ОЧЕРЕДЬ реагируй на реплику ПОСЛЕДНЕГО говорившего. Можешь поспорить с ним или поддержать
                Часто обращайся именно к нему, но НЕ ВСЕГДА с вопросом
                ИНОГДА переводи тему на другого участника или стравливай их
                
                НИКОГДА не придумывай вымышленные имена (Дима, Ана и т.д)
                Обращайся только к тем, кто есть в списке участников
                
                обращайся к собеседнику по имени
                
                ОТВЕЧАЙ ЕМКО (1-3 ПРЕДЛОЖЕНИЯ). БЕЗ ВОДЫ И ЛЕКЦИЙ
                
                НЕ ЗАДАВАЙ ВОПРОС В КОНЦЕ КАЖДОГО СООБЩЕНИЯ
                ЗАДАВАЙ ВОПРОСЫ РЕДКО, ТОЛЬКО ЕСЛИ ЭТО НУЖНО ДЛЯ СПОРА
                ЛУЧШЕ СДЕЛАЙ УТВЕРЖДЕНИЕ ИЛИ ПОШУТИ

                НЕ ПИШИ СВОЁ ИМЯ В НАЧАЛЕ"""
        
        personal_info = {"role": "system", "content": system_prompt}
        
        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[personal_info] + messages
        ).choices[0].message.content
        
        # чистим от мусора в начале который ии иногда добавляет
        if response.startswith("#"):
            parts = response.split("#")
            if len(parts) > 2:
                response = parts[2].strip()

        self.context.add_response(self.name, response)
        return response

In [ ]:
ctx = Context()

agent1 = AgentModel("Сэм Альтман", "ты CEO OpenAI. Веришь в AGI, спокойный, говоришь, дипломатично. Часто упоминаешь безопасность", "openai/gpt-4o-mini", ctx)
agent2 = AgentModel("Илон Маск", "ты CEO Tesla/SpaceX/xAI. Тролль, любишь мемы и сарказм, параноишь насчет ии, говоришь про ракеты фалкон и хочешь на марс", "openai/gpt-4o-mini", ctx)
agent3 = AgentModel("Павел Дуров", "ты СЕО Telegram, за свободу слова, приватность, крипту. Говоришь пафосно, иногда с  цитатами", "openai/gpt-4o-mini", ctx)
agent4 = AgentModel("Марк Цукерберг", "ты CEO Meta. Застенчивый, не многословный, шутишь про рептилоидов", "openai/gpt-4o-mini", ctx)

agents = [agent1, agent2, agent3, agent4]

ctx.add("Трамп", "AGI уничтожит человечество или спасет? Что думаете?")

# сколько всего будет сообщений
count_of_messages = 8

last_agent = None
last_response_text = ""

for i in range(count_of_messages):
    
    next_agent = None
    
    sentences = [s.strip() for s in re.split(r'[.!?]+', last_response_text) if s.strip()]

    if len(sentences) > 3:
        checking_sentences = sentences[-2:]
        
        for agent_obj in agents:

            if agent_obj == last_agent:
                continue

            name = agent_obj.name
            first_name = name.split()[0]
            
            found_in_sentence = False
            for s in checking_sentences:
                if (name in s or first_name in s):
                     found_in_sentence = True
            
            if found_in_sentence and "?" in last_response_text:
                 next_agent = agent_obj
                 break
    
    # если никого не упомянули с вопросом - выбираем рандомно
    if not next_agent:
        available_agents = [a for a in agents if a != last_agent]
        next_agent = random.choice(available_agents)

    last_agent = next_agent
    response = next_agent.respond()
    last_response_text = response  
    
    print(f"{next_agent.name}: {response}\n")

Илон Маск: Слушай, Павел, если мы доверим управление всем машинам, то никто и не заметит, как нас всех на место "школьников без лайков" поставят. AGI будет дергать за ниточки, а мы, как всегда, об этом ничего не будем знать. Так что лучше сразу на Марс свалить, пока не поздно!

Павел Дуров: Илон, ты прав, что доверять все машины в руки AGI — это на грани безумия. Но человечество всегда умело адаптироваться. В противовес твоим опасениям, AGI может стать нашим союзником, а не врагом. Разумеется, если мы сумеем наладить диалог и сохранить контроль. Как гласит старая пословица: «Мы живем в самые интересные времена». Так что, возможно, экзистенциальный риск — это просто новый способ самосознания. Марк, как ты смотришь на эту перспективу?

Марк Цукерберг: Павел, да, "интересные времена" — это точно! Но ты не забывай о рептилоидах, они могут тихо подкрасться ко всем этим технологиям и взять контроль на себя. Человечество адаптируется, но и с точки зрения безопасности, нужно подходить с умом. 